# Example of the Junior Notebook

In this Notebook, we present to ways to train the junior model. Either, with the simple Junior model or with an
advanced model that can be obtained, if some hyper-parameter tuning was conducted

In [1]:
import os
import logging
from pathlib import Path
import pickle
import json
import grid2op
import numpy as np
from lightsim2grid import LightSimBackend
from pathlib import Path
import tensorflow as tf

logging.basicConfig(level=logging.INFO)
example_path = os.getcwd()

Again check the environment

In [2]:
backend = LightSimBackend()
env_path = test_data_path = Path(example_path)/ "data" / "training_data_track1"
print(env_path)
env = grid2op.make(env_path, backend=backend)
env.set_id(1)
env.reset()
env.chronics_handler.get_name()
obs = env.get_obs()

## Simple Junior

Let's begin with the simple junior for training

In [3]:
from curriculumagent.junior.junior_student import train, validate

In [4]:
test_data_path = Path(example_path) / "data"
path1 = test_data_path / "actionspace_nminus1.npy"
path2 = test_data_path / "actionspace_tuples.npy"

action_set = [path1, path2]
model_path = Path(example_path) / "data" / "junior_model"
data_path = Path(example_path) / "data" / "junior_experience"

train(run_name="example",
      dataset_path=data_path,
      target_model_path=model_path,
      action_space_file=action_set,
      dataset_name='junior_dataset',
      epochs=100, seed=42)

In [5]:
validate(checkpoint_path=model_path,
         dataset_path=data_path,
         dataset_name='junior_dataset',
         action_space_file=action_set)

## Advanced Junior

The second option is a more advanced junior agent, that has selected hyper-parameters. You can gather these
hyperparameters by running a NNI or ```ray.tune``` experiment. The hyper-parameters need then to be supplied via a
dictionary. Note that it is also possible to add a scaler, e.g., a ```MinMaxScaler``` to the Junior. We advise for this \
        option, given that neural networks require values around [-1,1].

Note, for simplification we only load the best parameters into the Notebook.

In [21]:
with open(Path(example_path) / "data" / 'junior_best_params.json') as json_file:
    best_params = json.load(json_file)

best_params["epochs"] = 1
best_params

In [22]:
from curriculumagent.junior.junior_student import Junior
from curriculumagent.junior.junior_student import load_dataset
from sklearn.preprocessing import MinMaxScaler

Due to the fact that we also want to show the application of a scaler, we additionally show the whole pipeline of the   Junior agent. Therefore, we first fit the data with a MinMaxScaler:


In [23]:
s_train, a_train, s_validate, a_validate, _, _ = load_dataset(data_path, "junior_dataset")
scaler = MinMaxScaler()
s_tr_t = scaler.fit_transform(s_train)
s_val_t = scaler.transform(s_validate)

path1 = test_data_path / "actionspace_nminus1.npy"
path2 = test_data_path / "actionspace_tuples.npy"

action_set = [path1, path2]

In [24]:
# Pickle the scaler for later
with open( Path(example_path) / "data" / 'scaler_junior.pkl', "wb") as fp:
    pickle.dump(scaler, fp)

In [25]:
junior = Junior(config=best_params,
                        # Optimally, you know the number of actions:
                        action_space_file=action_set,
                        seed=42)
junior.model.compile()
#junior.model.summary()

In [26]:
history = junior.train(log_dir=model_path / "hyperparam",
                       ckpt_dir=model_path / "hyperparam",
                       patience=5,
                       x_train=s_tr_t, y_train=a_train,
                       x_validate=s_val_t, y_validate=a_validate)

In [27]:
junior.model.save(model_path / "hyperparam")